# **Import the Libraries**

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model

# **Call The Dataset**

In [ ]:
# !pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Open the Google Sheet by title
worksheet = gc.open("preprocessed_data").sheet1  # Replace "data" with the actual title of your Google Sheet

# Get all values from the sheet as a list of lists
data = worksheet.get_all_values()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])  # Assuming the first row contains column headers
df

,URL,Perfume Name,Gender,Brand,Description,Description Blockquote,Top Notes,Middle Notes,Base Notes,Image,...,violet,soapy,coca-cola,foresty,wine,woody,sand,herbal,mineral,white floral
0,https://www.fragrantica.com/perfume/Zara/007-F...,007 Feels Like Summer Zara,for women,Zara,007 Feels Like Summer by Zara is a Aromatic Fr...,,,,,https://fimgs.net/mdimg/perfume/375x500.60102.jpg,...,0,0,0,0,0,0.671873235,0,0,0,0
1,https://www.fragrantica.com/perfume/Tauer-Perf...,02 L'Air du Desert Marocain Tauer Perfumes,for women and men,Tauer Perfumes,02 L'Air du Desert Marocain by Tauer Perfumes ...,L`Air du Desert Marocain was made as a lighter...,"Coriander, Cumin, Petitgrain, Lavender","Labdanum, Birch, Jasmine, Geranium","Amber, Cedar, Vetiver, Patchouli, Oakmoss",https://fimgs.net/mdimg/perfume/375x500.4573.jpg,...,0,0,0,0,0,1,0,0.49095,0,0
2,https://www.fragrantica.com/perfume/Rabanne/1-...,1 Million Cologne Rabanne,for men,Rabanne,1 Million Cologne by Rabanne is a Woody Spicy...,In 2008 Paco Rabanne presented masculine fragr...,,,,https://fimgs.net/mdimg/perfume/375x500.30497.jpg,...,0,0,0,0,0,0.4900177857,0,0,0,0
3,https://www.fragrantica.com/perfume/Rabanne/1-...,1 Million Eau de Toilette Collector Edition Ra...,for men,Rabanne,1 Million Eau de Toilette Collector Edition by...,,,,,https://fimgs.net/mdimg/perfume/375x500.47198.jpg,...,0,0,0,0,0,0.7943979801,0,0,0,0
4,https://www.fragrantica.com/perfume/Rabanne/1-...,1 Million Privé Rabanne,for men,Rabanne,1 Million Privé by Rabanne is a Oriental Woody...,Paco Rabanne launched 1 Million fragrance in 2...,,,,https://fimgs.net/mdimg/perfume/375x500.37698.jpg,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20331,https://www.fragrantica.com/perfume/Rasasi/Yum...,Yumn Pour Homme Rasasi,for men,Rasasi,Yumn Pour Homme by Rasasi is a Oriental Woody ...,,,,,https://fimgs.net/mdimg/perfume/375x500.92106.jpg,...,0,0,0,0,0,0.835000614,0,0.4574982247,0,0
20332,https://www.fragrantica.com/perfume/Lush/Yuzu-...,Yuzu and Cocoa Lush,for women and men,Lush,Yuzu and Cocoa by Lush is a Aromatic fragrance...,,,,,https://fimgs.net/mdimg/perfume/375x500.60097.jpg,...,0,0,0,0,0,0,0,0,0,0
20333,https://www.fragrantica.com/perfume/Lush/Zesty...,Zesty Lush,for women and men,Lush,Zesty by Lush is a fragrance for women and men...,"Zesty body spray is a fresh, fragrant burst of...",,,,https://fimgs.net/mdimg/perfume/375x500.93841.jpg,...,0,0,0,0,0,0,0,0,0,0
20334,https://www.fragrantica.com/perfume/Rasasi/Zin...,Zinobia Rasasi,for women,Rasasi,Zinobia by Rasasi is a Oriental Floral fragran...,,,,,https://fimgs.net/mdimg/perfume/375x500.19664.jpg,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Ubah kolom Description menjadi lowercase
df['Description'] = df['Description'].str.lower()
df['Description'] = df['Description'].str.replace('\n', '', regex=False)

# **Build BERT Embedding**

Sentence Transformer

In [ ]:
# Load model pre-trained SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode dokumen dari kolom Description
doc_embeddings = model.encode(df['Description'].tolist())
doc_embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-0.12809119, -0.05978642,  0.04440703, ...,  0.01222474,
        -0.0280526 , -0.01361782],
       [-0.04182183, -0.04275753,  0.04450448, ...,  0.03269968,
         0.00937822,  0.07551304],
       [-0.01087738, -0.05494628, -0.00955697, ..., -0.05511973,
         0.03850576,  0.04883621],
       ...,
       [-0.06113426,  0.00807674,  0.07117152, ...,  0.02819667,
         0.1048659 ,  0.01607318],
       [-0.06846491,  0.05545558, -0.00315366, ..., -0.02289503,
         0.01505702,  0.01257933],
       [-0.05284953,  0.01214327,  0.06039776, ..., -0.00041361,
        -0.01342415,  0.00365134]], dtype=float32)

# **Compute Similarity**

In [ ]:
def recommend(query, doc_embeddings, df):
    # Encode query
    query_embedding = model.encode([query])

    # Hitung kesamaan cosine
    similarities = cosine_similarity(query_embedding, doc_embeddings)

    # Urutkan dokumen berdasarkan skor kesamaan
    ranked_indices = similarities.argsort()[0][::-1]

    # Ambil dokumen yang sesuai
    results = df.iloc[ranked_indices]
    results['similarity'] = similarities[0][ranked_indices]
    return results


# **Get Recommendations**

## Using Product Description In Dataset as Query

In [ ]:
index = 1
value_description = df.iloc[index, df.columns.get_loc('Description')].lower()
value_description

"02 l'air du desert marocain by tauer perfumes is a oriental spicy fragrance for women and men. 02 l'air du desert marocain was launched in 2005. the nose behind this fragrance is andy tauer. top notes are coriander, cumin, petitgrain and lavender; middle notes are labdanum, birch, jasmine and geranium; base notes are amber, cedar, vetiver, patchouli and oakmoss. l`air du desert marocain was made as a lighter version of andy's previous le maroc pour elle. one of the most popular in the line. the fragrance is composed of spicy oriental notes, inspired by saharan desert in the moonlight, a moroccan night.  the top is combined of coriander, cumin with a hint of petitgrain. the heart features rock rose and jasmine. the base includes cedar, vetiver and ambergris."

In [ ]:
results = recommend(value_description, doc_embeddings, df)
results

<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


,URL,Perfume Name,Gender,Brand,Description,Description Blockquote,Top Notes,Middle Notes,Base Notes,Image,...,soapy,coca-cola,foresty,wine,woody,sand,herbal,mineral,white floral,similarity
1,https://www.fragrantica.com/perfume/Tauer-Perf...,02 L'Air du Desert Marocain Tauer Perfumes,for women and men,Tauer Perfumes,02 l'air du desert marocain by tauer perfumes ...,L`Air du Desert Marocain was made as a lighter...,"Coriander, Cumin, Petitgrain, Lavender","Labdanum, Birch, Jasmine, Geranium","Amber, Cedar, Vetiver, Patchouli, Oakmoss",https://fimgs.net/mdimg/perfume/375x500.4573.jpg,...,0,0,0,0,1,0,0.49095,0,0,1.000000
1888,https://www.fragrantica.com/perfume/Tom-Ford/B...,Bois Marocain Tom Ford,for women and men,Tom Ford,bois marocain by tom ford is a woody spicy fra...,Private Blend: Bois Marocain by Tom Ford is a ...,,,,https://fimgs.net/mdimg/perfume/375x500.7942.jpg,...,0,0,0,0,1,0,0,0,0,0.727609
15553,https://www.fragrantica.com/perfume/Nina-Ricci...,L'Air du Temps Parfum Nina Ricci,for women,Nina Ricci,l'air du temps parfum by nina ricci is a flora...,,,,,https://fimgs.net/mdimg/perfume/375x500.67112.jpg,...,0,0,0,0,0.9327484201,0,0,0,0.6774489041,0.709303
15534,https://www.fragrantica.com/perfume/Nina-Ricci...,L'Air du Paradis Nina Ricci,for women,Nina Ricci,l'air du paradis by nina ricci is a floral fru...,,,,,https://fimgs.net/mdimg/perfume/375x500.49060.jpg,...,0,0,0,0,0,0,0,0,0.5924512541,0.708639
7739,https://www.fragrantica.com/perfume/Louis-Vuit...,Fleur du Désert Louis Vuitton,for women and men,Louis Vuitton,fleur du désert by louis vuitton is a oriental...,Louis Vuitton launches Fleur du Désert fragran...,,,,https://fimgs.net/mdimg/perfume/375x500.73006.jpg,...,0,0,0,0,0,0,0,0,1,0.705596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5946,https://www.fragrantica.com/perfume/Avon/Avon-...,Avon Sport for Her Vitality Avon,for women,Avon,avon sport for her vitality by avon is a flora...,,,,,https://fimgs.net/mdimg/perfume/375x500.31285.jpg,...,0,0,0,0,0,0,0,0,0,0.186679
10793,https://www.fragrantica.com/perfume/Bath-Body-...,Strawberry Pound Cake Bath & Body Works,for women,Bath & Body Works,strawberry pound cake by bath & body works is ...,,,,,https://fimgs.net/mdimg/perfume/375x500.64780.jpg,...,0,0,0,0,0,0,0,0,0,0.184361
4748,https://www.fragrantica.com/perfume/Amouage/Po...,Portrayal Woman Amouage,for women,Amouage,portrayal woman by amouage is a oriental flora...,The fifth chapter of the second cycle of the A...,Jasmine,"Tobacco, Vanilla",Elemi,https://fimgs.net/mdimg/perfume/375x500.54459.jpg,...,0,0,0,0,0,0,0,0,1,0.175498
8755,https://www.fragrantica.com/perfume/Serge-Lute...,L'orpheline Serge Lutens,for women and men,Serge Lutens,l'orpheline by serge lutens is a fragrance for...,"1. Yet another girl!\nYes, if you believe that...",,,,https://fimgs.net/mdimg/perfume/375x500.26214.jpg,...,0,0,0,0,0,0,0,0,0,0.078665


## User Created Query

In [ ]:
# Contoh penggunaan
query = "floral fragrance from Bath & Body Works for spring"

results = recommend(query, doc_embeddings, df)

# Print hasil pencarian
print(results.index[:15].tolist())
results.head(15)

[7789, 8873, 11530, 6693, 6951, 8870, 10433, 9157, 9348, 9379, 5910, 8030, 5868, 6461, 7974]


<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


,URL,Perfume Name,Gender,Brand,Description,Description Blockquote,Top Notes,Middle Notes,Base Notes,Image,...,soapy,coca-cola,foresty,wine,woody,sand,herbal,mineral,white floral,similarity
7789,https://www.fragrantica.com/perfume/Bath-Body-...,Floral Fantasy Bath & Body Works,for women,Bath & Body Works,floral fantasy by bath & body works is a flora...,,,,,https://fimgs.net/mdimg/perfume/375x500.93115.jpg,...,0,0,0,0,0.72,0,0,0,0.79999936,0.815794
8873,https://www.fragrantica.com/perfume/Bath-Body-...,Lavender & Spring Apricot Bath & Body Works,for women,Bath & Body Works,lavender & spring apricot by bath & body works...,,,,,https://fimgs.net/mdimg/perfume/375x500.38147.jpg,...,0,0,0,0,1,0,0,0,0,0.779729
11530,https://www.fragrantica.com/perfume/Bath-Body-...,Water Lily Springs Bath & Body Works,for women,Bath & Body Works,water lily springs by bath & body works is a f...,,,,,https://fimgs.net/mdimg/perfume/375x500.91504.jpg,...,0,0,0,0,0,0,0,0,0.8764699478,0.777843
6693,https://www.fragrantica.com/perfume/Bath-Body-...,Cherry Blossom Bath & Body Works,for women,Bath & Body Works,cherry blossom by bath & body works is a flora...,Cherry Blossom Body Mist by Bath & Body Works ...,,,,https://fimgs.net/mdimg/perfume/375x500.94696.jpg,...,0,0,0,0,0,0,0,0,0,0.773104
6951,https://www.fragrantica.com/perfume/Bath-Body-...,Cozy Season Bath & Body Works,for women,Bath & Body Works,cozy season by bath & body works is a woody ar...,,,,,https://fimgs.net/mdimg/perfume/375x500.77342.jpg,...,0,0,0,0,0.5437153354,0,0,0,0,0.751934
8870,https://www.fragrantica.com/perfume/Bath-Body-...,Lavender in Bloom Bath & Body Works,for women,Bath & Body Works,lavender in bloom by bath & body works is a ar...,,,,,https://fimgs.net/mdimg/perfume/375x500.64802.jpg,...,0,0,0,0,0.6733328597,0,0.4400007105,0,0.8833357017,0.744482
10433,https://www.fragrantica.com/perfume/Bath-Body-...,Sapphire Moon Bath & Body Works,for women,Bath & Body Works,sapphire moon by bath & body works is a floral...,,,,,https://fimgs.net/mdimg/perfume/375x500.78921.jpg,...,0,0,0,0,0,0,0,0,0,0.736280
9157,https://www.fragrantica.com/perfume/Bath-Body-...,Magnolia Charm Bath & Body Works,for women,Bath & Body Works,magnolia charm by bath & body works is a flora...,,,,,https://fimgs.net/mdimg/perfume/375x500.77762.jpg,...,0,0,0,0,0.4794873268,0,0,0,0,0.732759
9348,https://www.fragrantica.com/perfume/Bath-Body-...,Misty Morning Bath & Body Works,for women,Bath & Body Works,misty morning by bath & body works is a fragra...,,,,,https://fimgs.net/mdimg/perfume/375x500.74734.jpg,...,0,0,0,0,0,0,0,0,1,0.731393
9379,https://www.fragrantica.com/perfume/Bath-Body-...,Moonlight Magic Bath & Body Works,for women,Bath & Body Works,moonlight magic by bath & body works is a flor...,Moonlight Magic by Bath and Body Works is a fl...,,,,https://fimgs.net/mdimg/perfume/375x500.9804.jpg,...,0,0,0,0,0.5390708924,0,0,0,0.4912574765,0.725733


# **Evaluate The Result**

In [ ]:
# Function to calculate DCG
def dcg_at_k(actual, predicted, k=10):
    dcg = 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual:
            dcg += 1 / np.log2(i + 2)  # i+2 to handle rank starting at 1
    return dcg

# Function to calculate NDCG
def ndcg_at_k(actual, predicted, k=10):
    dcg = dcg_at_k(actual, predicted, k)
    idcg = dcg_at_k(actual, actual[:k])  # Ideal DCG
    return dcg / idcg if idcg > 0 else 0.0

# Function to calculate AP
def average_precision(actual, predicted, k=10):
    if not actual:
        return 0.0

    precision_values = []
    num_rel = 0
    for i, p in enumerate(predicted):
      if i >= k:
        break
      if p in actual:
        num_rel += 1
        precision_values.append(num_rel / (i + 1))

    if not precision_values:
      return 0.0
    return sum(precision_values) / min(len(actual),k)

# Function to calculate RR
def reciprocal_rank(actual, predicted, k=10):
    for i, item in enumerate(predicted):
      if i >= k:
        break
      if item in actual:
        return 1 / (i + 1)
    return 0.0


# Function to calculate Mean Metrics
def calculate_mean_metrics(df, queries, doc_embeddings, k=10):
    mean_ndcg = 0.0
    mean_ap = 0.0
    mean_rr = 0.0
    num_queries = len(queries)

    for query_text, relevant_indices in queries.items():
        # Get recommendations
        recommendations = recommend(query_text, doc_embeddings, df)

        # Ensure 'URL' column exists
        if 'URL' not in df.columns:
            raise KeyError("Kolom 'URL' tidak ditemukan dalam DataFrame.")

        # Get URLs from relevant indices and recommended results
        # Convert relevant_items to a list to enable slicing
        relevant_items = list(df.loc[relevant_indices, 'URL'])
        recommended_items = recommendations['URL'].tolist()

        # Calculate metrics for this query
        ndcg = ndcg_at_k(relevant_items, recommended_items, k)
        avg_prec = average_precision(relevant_items, recommended_items, k)
        recip_rank = reciprocal_rank(relevant_items, recommended_items, k)

        mean_ndcg += ndcg
        mean_ap += avg_prec
        mean_rr += recip_rank

        print(f"Query: {query_text}")
        print(f"  NDCG@{k}: {ndcg:.4f}")
        print(f"  Average Precision: {avg_prec:.4f}")
        print(f"  Reciprocal Rank: {recip_rank:.4f}")

    if num_queries == 0:
        return 0.0, 0.0, 0.0
    return mean_ndcg / num_queries, mean_ap / num_queries, mean_rr / num_queries


In [ ]:
# Define queries and relevant indices
queries = {
    "floral and vanilla for women": [17376, 18466, 11317, 4892, 1626, 7988, 2349, 7795, 20254, 7974],
    "gasoline and woody from diesel for men": [13676, 14707, 14703, 14705, 14699, 14708, 13675, 13683, 13682, 12344],
    "forest pine from axe" : [4543, 19531, 7903, 20303, 12118, 19403, 17560, 3784, 14685, 5825],
    "floral fragrance from Bath & Body Works for spring":[8873, 9812, 11530, 6693, 9157, 7789, 5868, 8030, 9379, 5910],
    "lemon scented hair perfume by Byredo": [6012, 8220, 6568, 6207, 8773, 9356, 10351, 15915, 6206, 17561],
    "coffee latte vanilla cream" : [19135, 3706, 4811, 8051, 5130, 6876, 11318, 6891, 11266, 12700],
    "cedar, musk and vetiver by Issey Miyake" : [15628, 10868, 17934, 15899, 15572, 16861, 11429, 16153, 7898, 11791],
    "white musk for women and men from The Body Shop": [18742, 18750, 18747, 18749, 12565, 18741, 18745, 18737, 18740, 18743],
    "green tea scent for women" : [14846, 15914, 14870, 14845, 17985, 10465, 14857, 14869, 14856, 15121,],
    "cherry blossom and pear for women": [5280, 8590, 8591, 6692, 2510, 2512, 9550, 8592, 2975, 14793]
    }

# Evaluate the model and print the results
mean_ndcg, mean_ap, mean_rr = calculate_mean_metrics(df, queries, doc_embeddings, k=10)
print("\n------------------------------------------\n")
print(f"Mean NDCG@10: {mean_ndcg:.4f}")
print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mean_rr:.4f}")

<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


Query: floral and vanilla for women
  NDCG@10: 0.4595
  Average Precision: 0.2444
  Reciprocal Rank: 1.0000
Query: gasoline and woody from diesel for men
  NDCG@10: 0.7020
  Average Precision: 0.5383
  Reciprocal Rank: 1.0000


<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


Query: forest pine from axe
  NDCG@10: 0.7917
  Average Precision: 0.6732
  Reciprocal Rank: 1.0000
Query: floral fragrance from Bath & Body Works for spring
  NDCG@10: 0.6968
  Average Precision: 0.5225
  Reciprocal Rank: 1.0000


<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


Query: lemon scented hair perfume by Byredo
  NDCG@10: 0.7788
  Average Precision: 0.6367
  Reciprocal Rank: 1.0000
Query: coffee latte vanilla cream
  NDCG@10: 0.5779
  Average Precision: 0.3484
  Reciprocal Rank: 1.0000


<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


Query: cedar, musk and vetiver by Issey Miyake
  NDCG@10: 0.6555
  Average Precision: 0.4492
  Reciprocal Rank: 1.0000
Query: white musk for women and men from The Body Shop
  NDCG@10: 0.7126
  Average Precision: 0.5600
  Reciprocal Rank: 1.0000
Query: green tea scent for women
  NDCG@10: 0.9337
  Average Precision: 0.8900
  Reciprocal Rank: 1.0000
Query: cherry blossom and pear for women
  NDCG@10: 0.8358
  Average Precision: 0.7068
  Reciprocal Rank: 1.0000

------------------------------------------

Mean NDCG@10: 0.7144
Mean Average Precision (MAP): 0.5570
Mean Reciprocal Rank (MRR): 1.0000


<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]


In [ ]:
def calculate_mean_metrics_with_dataframe(df, queries, doc_embeddings, k=10):
    # List untuk menyimpan hasil per-query
    results = []

    mean_ndcg = 0.0
    mean_ap = 0.0
    mean_rr = 0.0
    num_queries = len(queries)

    for query_text, relevant_indices in queries.items():
        # Get recommendations
        recommendations = recommend(query_text, doc_embeddings, df)

        # Get recommended indices
        recommended_indices = recommendations.index.tolist()

        # Calculate metrics for this query
        ndcg = ndcg_at_k(relevant_indices, recommended_indices, k)
        avg_prec = average_precision(relevant_indices, recommended_indices, k)
        recip_rank = reciprocal_rank(relevant_indices, recommended_indices, k)

        # Tambahkan hasil ke daftar
        results.append({
            "Query": query_text,
            "Expected Output": ", ".join(map(str, relevant_indices)),
            "Output": ", ".join(map(str, recommended_indices)),
            "NDCG": ndcg,
            "MAP": avg_prec,
            "MRR": recip_rank
        })

        # Akumulasi untuk menghitung rata-rata keseluruhan
        mean_ndcg += ndcg
        mean_ap += avg_prec
        mean_rr += recip_rank

    # Hitung rata-rata keseluruhan
    if num_queries > 0:
        mean_ndcg /= num_queries
        mean_ap /= num_queries
        mean_rr /= num_queries

    # Tambahkan baris untuk rata-rata keseluruhan ke hasil
    results.append({
        "Query": "Average",
        "Expected Output": "-",
        "Output": "-",
        "NDCG": mean_ndcg,
        "MAP": mean_ap,
        "MRR": mean_rr
    })

    # Buat DataFrame dari hasil
    results_df = pd.DataFrame(results)
    return results_df

# Panggil fungsi untuk menghitung metrik dan buat DataFrame hasil
results_df = calculate_mean_metrics_with_dataframe(df, queries, doc_embeddings, k=10)

# Tampilkan DataFrame
results_df

<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['similarity'] = similarities[0][ranked_indices]
<ipython-input-9-7cdfa9674ff7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,Query,Expected Output,Output,NDCG,MAP,MRR
0,floral and vanilla for women,"17376, 18466, 11317, 4892, 1626, 7988, 2349, 7...","17376, 11330, 18458, 18466, 5669, 7974, 18468,...",0.459533,0.244444,1.0
1,gasoline and woody from diesel for men,"13676, 14707, 14703, 14705, 14699, 14708, 1367...","13675, 13682, 13676, 13678, 13683, 14708, 1349...",0.701973,0.538333,1.0
2,forest pine from axe,"4543, 19531, 7903, 20303, 12118, 19403, 17560,...","19403, 7903, 20303, 4543, 12118, 19454, 19531,...",0.791727,0.673214,1.0
3,floral fragrance from Bath & Body Works for sp...,"8873, 9812, 11530, 6693, 9157, 7789, 5868, 803...","7789, 8873, 11530, 6693, 6951, 8870, 10433, 91...",0.696840,0.522500,1.0
4,lemon scented hair perfume by Byredo,"6012, 8220, 6568, 6207, 8773, 9356, 10351, 159...","6207, 6568, 8773, 8220, 6012, 10914, 19716, 94...",0.778807,0.636667,1.0
5,coffee latte vanilla cream,"19135, 3706, 4811, 8051, 5130, 6876, 11318, 68...","19135, 6954, 10181, 18470, 3706, 2349, 5130, 8...",0.577905,0.348413,1.0
6,"cedar, musk and vetiver by Issey Miyake","15628, 10868, 17934, 15899, 15572, 16861, 1142...","10868, 15628, 16610, 4554, 15899, 17934, 5348,...",0.655548,0.449167,1.0
7,white musk for women and men from The Body Shop,"18742, 18750, 18747, 18749, 12565, 18741, 1874...","18741, 18742, 18750, 18747, 18740, 18736, 1261...",0.712552,0.560000,1.0
8,green tea scent for women,"14846, 15914, 14870, 14845, 17985, 10465, 1485...","14846, 15914, 14870, 14845, 17985, 10465, 1485...",0.933746,0.890000,1.0
9,cherry blossom and pear for women,"5280, 8590, 8591, 6692, 2510, 2512, 9550, 8592...","5280, 2512, 6692, 3609, 8590, 2510, 9550, 1469...",0.835780,0.706825,1.0


from matplotlib import pyplot as plt
results_df['NDCG'].plot(kind='hist', bins=20, title='NDCG')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['MAP'].plot(kind='hist', bins=20, title='MAP')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df.plot(kind='scatter', x='NDCG', y='MAP', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['MRR']
  ys = series['NDCG']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_df.sort_values('MRR', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('MRR')
_ = plt.ylabel('NDCG')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['MRR']
  ys = series['MAP']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_df.sort_values('MRR', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('MRR')
_ = plt.ylabel('MAP')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['MRR']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'MRR'}, axis=1)
              .sort_values('MRR', ascending=True))
  xs = counted['MRR']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_df.sort_values('MRR', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('MRR')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
results_df['NDCG'].plot(kind='line', figsize=(8, 4), title='NDCG')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_df['MAP'].plot(kind='line', figsize=(8, 4), title='MAP')
plt.gca().spines[['top', 'right']].set_visible(False)

## **Save The Model and Embeddings**

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Save the model to Google Drive
model.save('/content/drive/My Drive/TA/bert_model')

# Save the embeddings as .npy file
np.save('/content/drive/My Drive/TA/doc_embeddings.npy', doc_embeddings)